# PSRO Sanity Checks
Run a tiny PSRO loop, then verify checkpoints and payoff shapes.

In [ ]:
import os
import sys
import shutil
from pathlib import Path

ROOT = Path(r"C:/Users/adidh/Documents/liars_poker")
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

from liars_poker.core import GameSpec
from liars_poker.algo.br_exact import best_response_exact
from liars_poker.training.psro import psro_loop

run_dir = ROOT / "artifacts" / "psro_runs" / "test_psro"
if run_dir.exists():
    shutil.rmtree(run_dir)


In [ ]:
spec = GameSpec(ranks=2, suits=2, hand_size=1, claim_kinds=("RankHigh",), suit_symmetry=True)

def oracle_first(spec, policy, debug=False):
    return best_response_exact(spec, policy, debug=debug)

def oracle_second(spec, policy, debug=False):
    return best_response_exact(spec, policy, debug=debug)

state, summary = psro_loop(
    spec,
    iterations=2,
    oracle_first=oracle_first,
    oracle_second=oracle_second,
    episodes_per_entry=200,
    run_dir=str(run_dir),
    debug=False,
)
print("pop sizes", len(state.pop_first), len(state.pop_second))
summary


In [ ]:
import json
import numpy as np

manifest_path = run_dir / "manifest.json"
stats_path = run_dir / "payoff_stats.npz"
logs_path = run_dir / "logs.jsonl"

assert manifest_path.exists()
assert stats_path.exists()
assert logs_path.exists()

manifest = json.loads(manifest_path.read_text(encoding="utf-8"))
npz = np.load(stats_path)

print("manifest iteration", manifest.get("iteration"))
print("payoff shapes", npz["mean"].shape, npz["n"].shape)

expected = 1 + 2  # initial + iterations
assert npz["mean"].shape == (expected, expected)

lines = logs_path.read_text(encoding="utf-8").strip().splitlines()
print("log lines", len(lines))
assert len(lines) == 2
